In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
import shutil
from sklearn.utils import shuffle
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import itertools
from IPython.display import Image, display, SVG
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from keras.models import Model,Sequential
from keras.layers import Input, LSTM, Dense, GRU, Dropout,TimeDistributed,Merge
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.layers import Bidirectional

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5110)


In [2]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Predicted label')
    plt.xlabel('True label')
    
    plt.show()
    
def filter_empty_entires(features, labels):
    valid_indexes = []
    
    nr_timesteps = features.shape[1]
    nr_features = features.shape[2]
    
    for index, feature in enumerate(features):
        elem_sum = np.sum(feature)
        if elem_sum != nr_timesteps*nr_features*-1:
            valid_indexes.append(index)
            
    return features[valid_indexes], labels[valid_indexes]
    
    
def get_mock_df(pred_df, data_channel_folder, round_decimals):
    
    data_channel_random_take_path = data_channel_folder + sorted(os.listdir(data_channel_folder))[4] # hack
    
    columns = read_df(data_channel_random_take_path, \
                      channel_columns[os.path.basename(data_channel_folder[:-1])], \
                      round_decimals,
                      no_drop_flag = False).columns
    
    mock_df = pd.DataFrame(index=pred_df.index.copy())

    for column in columns:
        mock_df[column] = 0.0

    return mock_df



def combine_features(channel_2_lists_dict):
    features_format = [f_format for channel, (f_format, _ )  in channel_2_lists_dict.items()]
    label_format = [l_format for channel, (_, l_format)  in channel_2_lists_dict.items()][0]
    
    features_format = np.concatenate(features_format,axis=2)
    
    return features_format, label_format
    

    
def scale_features(features_format, scaler = None):
    
    nr_examples, timesteps, nr_features = features_format.shape    
    reshaped_features = np.reshape(features_format,(nr_examples*timesteps,nr_features))
    
    if(scaler == None):
        scaler = MinMaxScaler() 
#         scaler = StandardScaler()
#         scaler = MaxAbsScaler()
#         scaler = RobustScaler()
        
        scaler = scaler.fit(reshaped_features)

    reshaped_features = scaler.transform(reshaped_features)
    features_format = np.reshape(reshaped_features,(nr_examples,timesteps,nr_features))
    
    return features_format, scaler


def standardize(f_df, l_df, timesteps, with_windows):
    
    data_timesteps = f_df.shape[1]
    
    max_timesteps = int(data_timesteps / timesteps) * timesteps

    if(max_timesteps == 0):
#         print("data_timesteps = {}".format(data_timesteps))
#         print("timesteps = {}".format(timesteps))
#         print("max_timesteps = {}".format(max_timesteps))

        f_matrix = f_df.as_matrix()
        f_matrix = pad_sequences(f_matrix, maxlen=timesteps, dtype='float64',padding='post', 
                                 truncating='post', value=-1333)
        f_matrix = np.expand_dims(np.transpose(f_matrix,(1,0)),axis=0)
        
        
        l_matrix = l_df.as_matrix()
        l_matrix = pad_sequences(l_matrix, maxlen=timesteps, dtype='float64',padding='post', 
                                 truncating='post', value=-1333)    
        l_matrix = np.expand_dims(np.transpose(l_matrix,(1,0)),axis=0)

        return f_matrix,l_matrix
    
    f_df = f_df.iloc[:,:max_timesteps]
    l_df = l_df.iloc[:,:max_timesteps]
        
    if with_windows:
        
        f_df_windows = []
        l_df_windows = []
        
        for window_start in range(max(1,max_timesteps - timesteps)):
            window_end = window_start + timesteps
            f_df_window = f_df.iloc[:,window_start:window_end]
            l_df_window = l_df.iloc[:,window_start:window_end]

            f_df_windows.append(f_df_window.T.as_matrix())
            l_df_windows.append(l_df_window.T.as_matrix())


        f_df_windows = np.stack(f_df_windows)
        l_df_windows = np.stack(l_df_windows)

        return f_df_windows, l_df_windows

    else:
        nr_splits = int(max_timesteps / timesteps)

        f_matrix = pd.concat(np.array_split(f_df,nr_splits)).T.as_matrix()
        l_matrix = pd.concat(np.array_split(l_df,nr_splits)).T.as_matrix()

        f_matrix_list = np.array_split(f_matrix,nr_splits)
        l_matrix_list = np.array_split(l_matrix,nr_splits)

        f_stacked_matrix = np.stack(f_matrix_list)
        l_stacked_matrix = np.stack(l_matrix_list)

        return f_stacked_matrix, l_stacked_matrix
    


def construct_format_data(joined_list, timesteps, with_windows):

    tuple_list = [standardize(features_df, labels_df, timesteps, with_windows) for features_df, labels_df in tqdm(joined_list)]
    
    features_list = [f for f,_ in tuple_list]
    label_list = [l for _,l in tuple_list]
    
    features_list = np.vstack(features_list)
    label_list = np.vstack(label_list)
        
    return features_list, label_list

def filter_zeros(df):
    df['sum_row'] = df.sum(axis=1)
    
    non_zero_index = -1

    for i,(_,row) in enumerate(df.iterrows()):
        if row['sum_row'] !=0 :
            non_zero_index = i
            break

    for index in range(len(df)):
        row = df.iloc[index]

        if(row['sum_row'] == 0):
            df.iloc[index] = df.iloc[non_zero_index].values
        else:
            non_zero_index = index
        
#     print("Number of zeros = {}".format(len(df[df['sum_row'] == 0])))
    
    df = df.drop(['sum_row'],axis=1)
    
    return df

def read_df(path, column_names, round_decimals, no_drop_flag):
    df = pd.read_csv(path, skiprows = 1, names = column_names)

    if(round_decimals != -1):
        df.loc[:,'Time'] = df.loc[:,'Time'].apply(lambda time: round(time,round_decimals))
    
    if(round_decimals == 0):
        df.loc[:,'Time'] = df.loc[:,'Time'].apply(lambda time: int(time))
        
    prev_len = len(df)
    df = df.drop_duplicates("Time")
    post_len = len(df)
    
    if(no_drop_flag):
        assert(prev_len == post_len)
    
    df = df.set_index("Time")
    
#     df = filter_zeros(df)
   
    
    return df


def get_data(base_path, data_channel, round_decimals, \
             label_folder, no_drop_flag, remove_extra_column, filter_agreement_score, \
             start_index, end_index):

    pred_folder = base_path + label_folder + "/"
    takes_names = sorted(os.listdir(pred_folder))

    joined_list = []
    
    if(end_index == None):
        end_index = len(takes_names)

    takes_names = takes_names[start_index:end_index]

    for take_name in tqdm(takes_names):

        pred_path = pred_folder + take_name
        data_channel_path = base_path + data_channel + "/" + take_name
                    
        pred_df = read_df(pred_path, \
                          channel_columns[label_folder],
                          round_decimals = round_decimals,
                          no_drop_flag = no_drop_flag)
        
        try:
            data_df = read_df(data_channel_path, \
                              channel_columns[data_channel],
                              round_decimals = round_decimals,
                              no_drop_flag = no_drop_flag)
            
        except Exception as e:
            data_channel_folder = base_path + data_channel + "/"
            data_df = get_mock_df(pred_df, data_channel_folder, round_decimals)
         
        joined_df = pred_df.join(data_df) #left join
        joined_df = joined_df.fillna(0.0)
    
#         joined_df = filter_zeros(joined_df)
    
        assert len(pred_df) == len(joined_df) 
        
        if(len(joined_df) <= timesteps):
                continue
                
        if filter_agreement_score:
            joined_df = joined_df[joined_df["Agreement score"] > 0.3]    
            if(len(joined_df) <= timesteps):
                continue
        
        joined_df = joined_df.T
        
        if remove_extra_column: # split features / labels df
            features_df = joined_df[(NR_LABELS+1):] # eliminate the 'Agreement Score'
        else:
            features_df = joined_df[NR_LABELS:]
        
        label_df = joined_df[:NR_LABELS]
            

        joined_list.append((features_df, label_df))
    

    return joined_list


def construct_channel_2_data_dict(base_path, channels, round_decimals, 
                                  timesteps, label_folder, no_drop_flag, remove_extra_column,
                                  with_windows,filter_agreement_score, start_index, end_index):
    channel_2_data_dict = {}
    for channel in channels:
        print("Channel = {}".format(channel))
        joined_list = get_data(base_path, channel, 
                               round_decimals = round_decimals, 
                               label_folder = label_folder, 
                               no_drop_flag = no_drop_flag,
                               remove_extra_column = remove_extra_column,
                               filter_agreement_score = filter_agreement_score,
                               start_index = start_index,
                               end_index = end_index) 
        
        print("construct_format_data....")
        features_list, label_list = construct_format_data(joined_list, timesteps, with_windows)
        
        channel_2_data_dict[channel] = (features_list, label_list)
    
    return channel_2_data_dict

def pipeline(start_index, end_index, with_windows, filter_agreement_score):
    
    # Construct data per channel
    channel_2_data_dict =  construct_channel_2_data_dict(train_base_path, \
                                                     channels = channels, \
                                                     round_decimals = round_decimals,
                                                     label_folder = 'labels',
                                                     timesteps = timesteps,
                                                     no_drop_flag = False,
                                                     remove_extra_column = True,
                                                     with_windows = with_windows,
                                                     filter_agreement_score = filter_agreement_score,
                                                     start_index = start_index,
                                                     end_index = end_index) 
    
    print("Constructed channel_2_data_dict...")
    
    for channel, (features, labels)  in channel_2_data_dict.items():
        print(channel)
        print(features.shape)
        print(labels.shape)
        
        
    # Combine channels
    
    features, labels = combine_features(channel_2_data_dict)
    
    features, labels = shuffle(features,labels,random_state=0)
    print(features.shape)
    print(labels.shape)
    
    return features, labels

def bring_to_standard_format(features,labels,test_timesteps):
    nr_splits = int(test_timesteps / timesteps)

    features_list = np.array_split(features,nr_splits,axis=1)
    label_list = np.array_split(labels,nr_splits,axis=1)

    features = np.vstack(features_list)
    labels = np.vstack(label_list)

    return features,labels

def filter_entries_with_valid_data(features,labels):
    min_wrong_index = [i for i,f in enumerate(features) if -1333 in f][0]
    
    features = features[:min_wrong_index]
    labels = labels[:min_wrong_index]
    
    return features, labels
        
def round_preds(preds):

    rounded_preds = np.zeros(preds.shape).astype(np.uint8)
    for index, pred in enumerate(preds):
        max_pred_index = np.argmax(pred)
        rounded_preds[index][max_pred_index] = 1
        
    return rounded_preds       


# Data

In [3]:
channel_columns = {}
channel_columns['audio'] = "Time,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,mfcc2_sma3,mfcc3_sma3,mfcc4_sma3,F0semitoneFrom27.5Hz_sma3nz,jitterLocal_sma3nz,shimmerLocaldB_sma3nz,HNRdBACF_sma3nz,logRelF0-H1-H2_sma3nz,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3amplitudeLogRelF0_sma3nz,pcm_fftMag_mfcc[0],pcm_fftMag_mfcc[1],pcm_fftMag_mfcc[2],pcm_fftMag_mfcc[3],pcm_fftMag_mfcc[4],pcm_fftMag_mfcc[5],pcm_fftMag_mfcc[6],pcm_fftMag_mfcc[7],pcm_fftMag_mfcc[8],pcm_fftMag_mfcc[9],pcm_fftMag_mfcc[10],pcm_fftMag_mfcc[11],pcm_fftMag_mfcc[12]".split(",")
channel_columns['eyes'] = "Time,Eyes_F_1,Eyes_F_2,Eyes_F_3,Eyes_F_4,Eyes_F_5,Eyes_F_6".split(",")
channel_columns['face_nn'] = ["Time"] + ["Face_F_"+str(i) for i in range(100)]
channel_columns['kinect'] = "Time,Dist_hands,Dist_LH_hip_center,Dist_RH_hip_center,Dist_spine_hip,Dist_LSh_head,Dist_RSh_head,Dist_spine_head,Dist_LH_spine,Dist_RH_spine,Dist_LH_head,Dist_RH_head,Symmetry_hands,RH_velocity,RH_Smoothness_Index,RH_jerk,RH_Curvature_Index,RH_acceleration,LH_velocity,LH_Smoothness_Index,LH_jerk,LH_Curvature_Index,LH_acceleration,Kinetic_Energy,Head_velocity,Head_jerk,Head_acceleration,Density_Index".split(",")
channel_columns['labels'] = "Time,Anger,Sad,Disgust,Happy,Scared,Neutral,Agreement score".split(",")
channel_columns['prediction'] = "Time,Anger,Sad,Disgust,Happy,Scared,Neutral".split(",")
channel_columns['features'] = "Anger,Sad,Disgust,Happy,Scared,Neutral".split(",")

NR_LABELS = 6

train_base_path = "./data/train/"
test_base_path = "./data/test/"

# Params

In [5]:
channels = ['audio', 'eyes', 'kinect', 'face_nn']
# channels = ['face_nn']

round_decimals = 0 # 0 -> 2
timesteps = 10

In [6]:
nr_examples = len(os.listdir(train_base_path+"labels"))

train_percentage = 0.7
barrier = int(nr_examples * train_percentage)

print(nr_examples)
print(barrier)

312
218


In [7]:
train_start = 0
train_end = barrier

test_start = barrier + 1
test_end = None

# train_start = 0
# train_end = 5

# test_start = 5 
# test_end = 6

In [8]:
train_features, train_labels = pipeline(train_start, 
                                        train_end, 
                                        with_windows = False,
                                        filter_agreement_score = False)

test_features, test_labels = pipeline(test_start, test_end,
                                        with_windows = False,
                                        filter_agreement_score = False,
                                     )


  1%|          | 2/218 [00:00<00:12, 17.28it/s]

Channel = audio


 10%|█         | 22/217 [00:00<00:00, 212.46it/s]

construct_format_data....


  2%|▏         | 4/218 [00:00<00:05, 39.15it/s]

...

Channel = face_nn


 23%|██▎       | 21/92 [00:00<00:00, 209.90it/s]

construct_format_data....


100%|██████████| 92/92 [00:00<00:00, 243.62it/s]

Constructed channel_2_data_dict...
kinect
(320, 10, 27)
(320, 10, 6)
eyes
(320, 10, 6)
(320, 10, 6)
audio
(320, 10, 36)
(320, 10, 6)
face_nn
(320, 10, 100)
(320, 10, 6)
(320, 10, 169)
(320, 10, 6)


In [9]:
print(train_features.shape)
print(test_features.shape)

print(train_labels.shape)
print(test_labels.shape)

(765, 10, 169)
(320, 10, 169)
(765, 10, 6)
(320, 10, 6)


# Scale Features

In [10]:
train_features,scaler = scale_features(train_features)
test_features,_ = scale_features(test_features,scaler)

print(train_features.shape)
print(test_features.shape)

(765, 10, 169)
(320, 10, 169)


# Augmentation

In [11]:
# reversed_train_features = np.flip(train_features,axis=1)
# reversed_train_labels = np.flip(train_labels,axis=1)

# train_features = np.vstack([train_features,reversed_train_features])
# train_labels = np.vstack([train_labels,reversed_train_labels])

# print(train_features.shape)
# print(train_labels.shape)

In [12]:
def cross_product(features):
    first_halves = features[:,:timesteps/2,:]
    second_halves = features[:,timesteps/2:,:]
    
    print(first_halves.shape)
    print(second_halves.shape)
    
    first_halves_list = list(first_halves)
    second_halves_list = list(second_halves)

    print(len(first_halves_list))
    print(len(second_halves_list))
    
    cross_product = list(itertools.product(first_halves_list,second_halves_list))

    print(len(cross_product))
    
    merged_back = np.stack([np.vstack([a,b]) for a,b in tqdm(cross_product)])
    
    return merged_back

In [13]:
train_features = cross_product(train_features)
train_labels = cross_product(train_labels)

print(train_features.shape)
print(train_labels.shape)

  0%|          | 0/585225 [00:00<?, ?it/s]

(765, 5, 169)
(765, 5, 169)
765
765
585225


  3%|▎         | 18645/585225 [00:00<00:03, 186418.18it/s]

(765, 5, 6)
(765, 5, 6)
765
765
585225


100%|██████████| 585225/585225 [00:02<00:00, 225462.00it/s]


(585225, 10, 169)
(585225, 10, 6)


# Data Split

In [ ]:
# def data_split(features):
    
#     kinect_f = features[...,:27]
#     eyes_f = features[...,27:33]
#     audio_f = features[...,33:69]
#     face_nn_f = features[...,69:]

#     return kinect_f, eyes_f, audio_f, face_nn_f

In [ ]:
# train_kinect_f, train_eyes_f, train_audio_f, train_face_nn_f = data_split(train_features)
# test_kinect_f, test_eyes_f, test_audio_f, test_face_nn_f = data_split(test_features)

# print(train_kinect_f.shape)
# print(train_eyes_f.shape)
# print(train_audio_f.shape)
# print(train_face_nn_f.shape)
# print("----")
# print(test_kinect_f.shape)
# print(test_eyes_f.shape)
# print(test_audio_f.shape)
# print(test_face_nn_f.shape)

# Network V0

In [ ]:
# def get_lstm(features_shape):
#     input_layer = Input(shape= (features_shape[1],features_shape[2]))
#     x = GRU(16,return_sequences=True)(input_layer)
#     x = TimeDistributed(Dense(128, activation='relu'))(x)    
#     x = Dropout(0.5)(x)
#     x = TimeDistributed(Dense(64, activation='relu'))(x)
#     x = Dropout(0.5)(x)
#     model = Model(input_layer, x)
    
#     return model


# def get_conv1d(features_shape):
#     input_layer = Input(shape= (features_shape[1],features_shape[2]))
#     x = Conv1D(64,3,activation='relu', border_mode='same')(input_layer)
#     x = TimeDistributed(Dense(128, activation='relu'))(x)    
#     x = Dropout(0.5)(x)
#     x = TimeDistributed(Dense(64, activation='relu'))(x)
#     x = Dropout(0.5)(x)
#     model = Model(input_layer, x)
    
#     return model



In [ ]:
# kinect_lstm = get_conv1d(train_kinect_f.shape)
# eyes_lstm = get_conv1d(train_eyes_f.shape)
# audio_lstm = get_conv1d(train_audio_f.shape)
# face_nn_lstm = get_conv1d(train_face_nn_f.shape)

# lstm_list = [kinect_lstm, eyes_lstm, audio_lstm, face_nn_lstm]

In [ ]:
# model = Sequential()
# # model.add(Merge(lstm_list, mode='concat'))
# model.add(Merge(lstm_list, mode='sum'))

# model.add(TimeDistributed(Dense(32, activation = 'relu')))

# model.add(TimeDistributed(Dense(NR_LABELS, activation = 'softmax')))
# model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# figure = SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))
# display(figure)


In [ ]:
# model.fit([train_kinect_f, train_eyes_f, train_audio_f, train_face_nn_f], train_labels,
#           nb_epoch = 200,
#           batch_size = 128,
#           validation_data = ([test_kinect_f, test_eyes_f, test_audio_f, test_face_nn_f], test_labels),
#           callbacks = [
#                   CSVLogger("./training.txt"),
#                   ModelCheckpoint("./models/model_temp.h5", monitor='val_acc', verbose= 1, save_best_only=True, mode='max')
#          ]
#          )

In [ ]:
# model.save_weights("./models/all_model_split.h5")

In [ ]:
# model.load_weights("./models/model_temp.h5")

In [ ]:
# train_acc = model.evaluate([train_kinect_f, train_eyes_f, train_audio_f, train_face_nn_f],train_labels)[1]
# test_acc = model.evaluate([test_kinect_f, test_eyes_f, test_audio_f, test_face_nn_f],test_labels)[1]

# print("\n\nTrain acc = {}".format(train_acc))
# print("Test acc = {}".format(test_acc))

# Network simple

In [ ]:
# # train_features = resh(train_features)
# # train_labels = resh(train_labels)
# # test_features = resh(test_features)
# # test_labels = resh(test_labels)


# print(train_features.shape)
# print(train_labels.shape)
# print(test_features.shape)
# print(test_labels.shape)

In [ ]:
# input_layer = Input(shape= (169,))
# x = Dense(128, activation='relu')(input_layer)    
# x = Dropout(0.5)(x)
# x = Dense(64, activation='relu')(x)
# x = Dropout(0.5)(x)

# pred_layer = Dense(NR_LABELS, activation='softmax')(x)

# model = Model(input_layer, pred_layer)
# model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(train_features, train_labels,
#           nb_epoch = 500,
#           batch_size = 128,
#           validation_data = (test_features, test_labels),
#           callbacks = [
#                   CSVLogger("./training.txt"),
#                   ModelCheckpoint("./models/model_temp.h5", monitor='val_acc', verbose= 1, save_best_only=True, mode='max')
#          ]
#          )

# Network sklearn

In [ ]:
# def resh(f):
#     return np.reshape(f,(f.shape[0],f.shape[1]*f.shape[2]))

# def sklearn_predict(clf,features):
#     test_pred = clf.predict(features)
#     test_preds_ohe = np.zeros((len(features),6))

#     for i,pred in enumerate(test_pred):
#         test_preds_ohe[i][pred] = 1
    
#     return test_preds_ohe

In [ ]:
# train_features = resh(train_features)
# train_labels = resh(train_labels)
# test_features = resh(test_features)
# test_labels = resh(test_labels)


# print(train_features.shape)
# print(train_labels.shape)
# print(test_features.shape)
# print(test_labels.shape)

In [ ]:
# train_labels_nr = np.array([np.argmax(l) for l in train_labels])
# test_labels_nr = np.array([np.argmax(l) for l in test_labels])

# print(train_labels_nr.shape)
# print(test_labels_nr.shape)


In [ ]:
# clf = RandomForestClassifier(n_estimators=100, verbose=1, n_jobs=24)
# # clf = GradientBoostingClassifier(n_estimators = 500, verbose = 1)

In [ ]:
# clf.fit(train_features, train_labels_nr)

In [ ]:
# test_preds_ohe = sklearn_predict(clf,test_features)
# accuracy_score(test_labels, test_preds_ohe)

In [ ]:
# train_preds_ohe = test_preds_ohe = sklearn_predict(clf,train_features)
# accuracy_score(train_labels, train_preds_ohe)

# Network V1

In [14]:
input_layer = Input(shape= (train_features.shape[1],train_features.shape[2]))
x = GRU(16,return_sequences=True)(input_layer)

x = TimeDistributed(Dense(128, activation='relu'))(x)
x = Dropout(0.5)(x)
x = TimeDistributed(Dense(64, activation='relu'))(x)
x = Dropout(0.5)(x)

pred_layer = TimeDistributed(Dense(NR_LABELS, activation='softmax'))(x)

model = Model(input_layer, pred_layer)
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
# model.summary()

In [16]:
model.fit(train_features, train_labels,
          nb_epoch = 100,
          batch_size = 128,
          validation_data = (test_features, test_labels),
          callbacks = [
                  CSVLogger("./training.txt"),
                  ModelCheckpoint("./models/model_temp.h5", monitor='val_acc', verbose= 1, save_best_only=True, mode='max')
         ]
         )

Train on 585225 samples, validate on 320 samples
Epoch 1/100
585225/585225 [==============================] - 68s - loss: 0.7976 - acc: 0.7143 - val_loss: 1.5298 - val_acc: 0.5694
Epoch 2/100
585225/585225 [==============================] - 66s - loss: 0.4715 - acc: 0.8404 - val_loss: 2.0199 - val_acc: 0.5700
Epoch 3/100
585225/585225 [==============================] - 66s - loss: 0.3726 - acc: 0.8756 - val_loss: 2.2562 - val_acc: 0.5703
Epoch 4/100
585225/585225 [==============================] - 64s - loss: 0.3162 - acc: 0.8953 - val_loss: 2.6069 - val_acc: 0.5659
Epoch 5/100
585225/585225 [==============================] - 65s - loss: 0.2798 - acc: 0.9080 - val_loss: 2.9409 - val_acc: 0.5641
Epoch 6/100
585225/585225 [==============================] - 65s - loss: 0.2526 - acc: 0.9174 - val_loss: 3.1307 - val_acc: 0.5522
Epoch 7/100
585225/585225 [==============================] - 65s - loss: 0.2317 - acc: 0.9247 - val_loss: 3.4192 - val_acc: 0.5175
Epoch 8/100
585225/585225 [=======

KeyboardInterrupt: 

In [ ]:
# save weights
# model.save_weights("./models/model_last_epoch.h5")
# model.save_weights("./models/scaled_augmented_acc=70.h5")
# model.save_weights("./models/all_scaled_decimals=0_lstm=16_v3.h5")
model.save_weights("./models/all_scaled_decimals=0_gru=16_epoch=600.h5")


In [ ]:
# load weights

# model.load_weights("./models/model_last_epoch.h5")
# model.load_weights("./models/model_temp.h5")

# model.load_weights("./models/scaled_augmented_acc=70.h5")
# model.load_weights("./models/all_scaled_decimals=0_gru=16_epoch=372.h5")
# model.load_weights("./models/scaled_decimals=0_gru=16_acc=61.h5")
# model.load_weights("./models/all_scaled_decimals=0_lstm=16_v3.h5")

model.load_weights("./models/all_scaled_decimals=0_gru=16_epoch=200.h5")

In [ ]:
train_acc = model.evaluate(train_features,train_labels)[1]
test_acc = model.evaluate(test_features,test_labels)[1]

print("\n\nTrain acc = {}".format(train_acc))
print("Test acc = {}".format(test_acc))

# Prediction

In [ ]:
def make_prediction(base_path, label_folder, is_train = False):
    
    prediction_path = base_path + "/" + label_folder + "/"
    nr_examples = len(os.listdir(prediction_path))
    take_names = sorted(os.listdir(prediction_path))

    test_timesteps = 10000
    
    pred_df_2_take_name_list = []
    
    for csv_index in tqdm(range(nr_examples)):

        test_channel_2_data_dict =  construct_channel_2_data_dict(base_path, 
                                                         channels = channels, 
                                                         round_decimals = round_decimals, 
                                                         label_folder = label_folder,
                                                         timesteps = test_timesteps,
                                                         no_drop_flag = False,
                                                         remove_extra_column = is_train,
                                                         with_windows = False,
                                                         filter_agreement_score = False,
                                                         start_index = csv_index,
                                                         end_index = csv_index + 1) 

        
        features, labels = combine_features(test_channel_2_data_dict)

        features, labels = bring_to_standard_format(features,labels, test_timesteps)

        features, labels = filter_entries_with_valid_data(features,labels)

        features, _ = scale_features(features, scaler = scaler)
        
#         test_kinect_f, test_eyes_f, test_audio_f, test_face_nn_f = data_split(features)
#         preds = model.predict([test_kinect_f, test_eyes_f, test_audio_f, test_face_nn_f])
       
        preds = model.predict(features)
        
        preds = preds.reshape(preds.shape[0] * preds.shape[1], preds.shape[2])
        
        preds = round_preds(preds)

        take_name = take_names[csv_index]
    
        ########
        
        label_df = pd.read_csv(prediction_path + take_name, dtype=str)[['Time']]
        label_df.loc[:,'Time_Bucket'] = label_df.loc[:,'Time'].apply(lambda time: round(float(time),round_decimals))

        buckets = set(label_df['Time_Bucket'])
        nr_buckets = len(buckets)
        pred_length = len(preds)

        preds_padded = np.zeros((nr_buckets,NR_LABELS))
        preds_padded[:pred_length,:] = preds

        last_pred_value = preds[-1]
        preds_padded[pred_length:,:] = last_pred_value

        bucket_2_pred = {bucket:pred for bucket,pred in zip(buckets,preds_padded)}

        label_df.loc[:,'pred'] = label_df.loc[:,'Time_Bucket'].apply(lambda bucket: bucket_2_pred[bucket])

        label_df[channel_columns['features']] = pd.DataFrame(label_df['pred'].values.tolist())
        label_df = label_df.drop(["Time_Bucket","pred"],axis=1)

        #######
        
        for col in channel_columns['features']:
            label_df[col] = label_df[col].astype(int)

        pred_df_2_take_name_list.append((label_df,take_name))
    
    return pred_df_2_take_name_list

In [ ]:
# pred_df_2_take_name_list = make_prediction(train_base_path, label_folder = 'labels', is_train = True)

In [ ]:
pred_df_2_take_name_list = make_prediction(test_base_path, label_folder = 'prediction')

# Write to disk

In [ ]:
output_path = "./prediction/"

shutil.rmtree(output_path)
os.mkdir(output_path)

In [ ]:
for pred_df, take_name in tqdm(pred_df_2_take_name_list):    
    pred_df.to_csv(output_path + take_name,index=False)

In [ ]:
%%bash
rm prediction.zip 

# Prediction Validity

In [ ]:
test_folder = "./data/test/prediction/"
pred_folder = "./prediction/"

test_paths = sorted(os.listdir(test_folder))
pred_paths = sorted(os.listdir(pred_folder))

In [ ]:
len(set(test_paths).difference(set(pred_paths)))

In [ ]:
for path in tqdm(test_paths):
    test_df = pd.read_csv(test_folder + path,dtype=str)
    pred_df = pd.read_csv(pred_folder + path,dtype=str)
    
    matrix = pred_df.drop("Time",axis=1).apply(pd.to_numeric).as_matrix()
    
    nr_rows = matrix.shape[0]
    matrix_sum = np.sum(matrix)

#     print("{} -> {}".format(nr_rows,matrix_sum))

    assert(len(set(test_df.index).difference(set(pred_df.index))) == 0)
    assert(list(test_df.index) == list(pred_df.index))
    assert(test_df.shape == pred_df.shape)
    assert(nr_rows == matrix_sum)

# Check Prediction Accuracy

In [ ]:
def compute_accuracy_on_train_set(pred_df_2_take_name_list):
    
    global_pred_vector = []
    global_gt_vector = []
    for pred_df, take_name in tqdm(pred_df_2_take_name_list):

        pred_df = pred_df.drop(['Time'],axis = 1)
        gt_df = pd.read_csv(train_base_path + "labels/" + take_name, dtype=str)\
                    .drop(["Time","Agreement score"],axis=1)

        pred_matrix = pred_df.as_matrix()
        gt_matrix = gt_df.as_matrix()

        pred_vector = [np.argmax(x) for x in pred_matrix]
        gt_vector = [np.argmax(x) for x in gt_matrix]

        global_pred_vector += pred_vector
        global_gt_vector += gt_vector
        
    return global_gt_vector, global_pred_vector


In [ ]:
gt_vector, pred_vector = compute_accuracy_on_train_set(pred_df_2_take_name_list)

In [ ]:
acc = accuracy_score(gt_vector, pred_vector)        
print("Train acc = {}".format(acc))

In [ ]:
cnf_matrix = confusion_matrix(gt_vector,pred_vector)
plot_confusion_matrix(cnf_matrix, classes = channel_columns['features'])

In [ ]:
pd.DataFrame([channel_columns['features'][i] for i in gt_vector],columns=['gt']).groupby('gt').size()

In [ ]:
pd.DataFrame([channel_columns['features'][i] for i in pred_vector],columns=['pred']).groupby('pred').size()

# Replace 0s

In [ ]:
import pandas as pd

In [ ]:
csv_path = "./data/train/audio/id1023e63c.csv"

In [ ]:
df = pd.read_csv(csv_path, skiprows = 1, names = channel_columns['audio'])
df = df.set_index("Time")

In [ ]:
df.head()